In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import os
import random

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy



/home/heisenberg/anaconda3/envs/RL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Explore the Spaces

In [2]:
Discrete(3)

Discrete(3)

In [5]:
print(Discrete(3).sample())
print(Discrete(3).sample())

0
2


In [6]:
Box(0,1, shape=(3,3))

Box([[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], (3, 3), float32)

In [8]:
Box(0,1, shape=(3,3)).sample()

array([[0.94578344, 0.76487947, 0.4459085 ],
       [0.62333924, 0.8586406 , 0.60429865],
       [0.8265448 , 0.27512774, 0.6257228 ]], dtype=float32)

In [9]:
Box(0,1, shape=(3,)).sample()

array([0.4730486 , 0.66486526, 0.9345994 ], dtype=float32)

In [10]:
Tuple((Discrete(3) , Box(0,1, shape=(3,)))).sample()

(1, array([0.3951616 , 0.28509513, 0.78273046], dtype=float32))

In [11]:
Dict({'height':Discrete(3) ,'speed': Box(0,1, shape=(3,))}).sample()

OrderedDict([('height', 2),
             ('speed',
              array([0.37094456, 0.9452412 , 0.63419867], dtype=float32))])

In [12]:
MultiBinary(4).sample()

array([0, 0, 1, 1], dtype=int8)

In [15]:
print(MultiDiscrete([5,2,100]).sample())
print(MultiDiscrete([5,2,100]).sample())
print(MultiDiscrete([5,2,100]).sample())

[ 2  0 33]
[ 4  0 99]
[ 3  0 31]


### Building an Environment
* Build an Agent to give us the best shower possible
* Randomly Temperature Fluctuates
* We want temp b/w 37 & 39

In [19]:
Box(0 , 100, shape = (1,)).sample()

array([90.41883], dtype=float32)

In [69]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3) # Turn temp up, down or keep it as it is
        self.observation_space = Box(0 , 100, shape=(1,)) # temp can be from 0 to 100
        self.state = 38 + random.randint(-3, 3) # initial state
        self.shower_length = 60
        

    def step(self, action):
        done = False
        self.state += action #- 1
        self.shower_length -= 1

        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}
        return self.state, reward, done, info


    def render(self):
        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state

In [70]:
env = ShowerEnv()

In [71]:
env.observation_space.sample()

array([69.9859], dtype=float32)

### Test our Environment

In [72]:
env.reset()


array([39.])

In [83]:
print(env.state)
curr_act = env.action_space.sample()
print(curr_act)
obs, reward, done, info = env.step(curr_act)
print(obs, reward, done, info)
print(env.shower_length)

[48.]
2
[50.] -1 False {}
49


In [84]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} ended with score {score}")

env.close()

Episode 1 ended with score -60
Episode 2 ended with score -58
Episode 3 ended with score -60
Episode 4 ended with score -60
Episode 5 ended with score -54


### Train Model

In [85]:
log_path = os.path.join('./train-artifacts', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log= log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [86]:
model.learn(total_timesteps=50000)

Logging to ./train-artifacts/logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -56.9    |
| time/              |          |
|    fps             | 454      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -57         |
| time/                   |             |
|    fps                  | 447         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007611882 |
|    clip_fraction        | 0.061       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.00

In [87]:
save_path = os.path.join('./train-artifacts','saved_models','CustomEnvBasic')
model.save(save_path)

In [89]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/home/heisenberg/anaconda3/envs/RL/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-60.0, 0.0)